In [57]:
import requests
from datetime import *
import time
from urllib.parse import unquote
from urllib.parse import quote_plus
from urllib.parse import unquote_plus
from urllib.parse import urlencode


url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

serviceKey="GdenAOBzMnIcPEJ6tyQqFbbSrAB%2B81vv7NDRBKe%2FbjomnxP5MpRkqe877BlZoLFm%2FqF35QGJcXvzteALmxDurg%3D%3D"
serviceKeyDecoded = unquote(serviceKey, 'UTF-8')

now = datetime.now()
today = datetime.today().strftime("%Y%m%d")
y = date.today() - timedelta(days=1)
yesterday = y.strftime("%Y%m%d")
nx = 60 # 위도와 경도를 x,y좌표로 변경
ny = 127

if now.hour>=23 and now.minute >10: # base_time와 base_date 구하는 함수
    base_time = "2310"
    base_date = today
else : 
    base_time = "2310"
    base_date = yesterday

In [229]:
# queryParams = '?' + urlencode({ quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date,
#                                 quote_plus('base_time') : base_time, quote_plus('nx') : nx, quote_plus('ny') : ny,
#                                 quote_plus('dataType') : 'json', quote_plus('numOfRows') : '289'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=120으로 설정해주었다
#                                 # 기상데이터는 12개로 12단위로 곱해서 row값을 정할 것, 24시간으로 하면 +1(최고기온떄문에)

# # 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
# res = requests.get(url + queryParams, verify=False) # verify=False이거 안 넣으면 에러남ㅜㅜ
# items = res.json().get('response').get('body').get('items') #데이터들 아이템에 저장
# # print(items)# 테스트

weather_data = dict()

# dictionary value값에 새로운 값을 더하도록 설계
for item in items['item']:
    # 예보 날짜 및 시간
    if item['category'] == 'TMP':
        try: 
            weather_data['fcstDate'] += [item['fcstDate']]
        except:
            weather_data['fcstDate'] = [item['fcstDate']]
    if item['category'] == 'TMP':
        try:
            weather_data['fcstTime'] += [item['fcstTime']]
        except:
            weather_data['fcstTime'] = [item['fcstTime']]
    # 기온
    if item['category'] == 'TMP':
        try:
            weather_data['tmp_hour'] += [item['fcstValue']]
        except:
            weather_data['tmp_hour'] = [item['fcstValue']]
    if item['category'] == 'TMN':
        try:
            weather_data['tmp_low'] += [weather_data['tmp_low'][0]]
        except:
            weather_data['tmp_low'] = [item['fcstValue']]
    if item['category'] == 'TMX':
        try:
            weather_data['tmp_high'] += [weather_data['tmp_high'][0]]
        except:
            weather_data['tmp_high'] = [item['fcstValue']]
    # 습도
    if item['category'] == 'REH':
        try:
            weather_data['hum'] += [item['fcstValue']]
        except:
            weather_data['hum'] = [item['fcstValue']]
    # 하늘상태: 맑음(1) 구름많은(3) 흐림(4)
    if item['category'] == 'SKY':
        try:
            weather_data['sky'] += [item['fcstValue']]
        except:
            weather_data['sky'] = [item['fcstValue']]
    # 1시간 동안 강수량
    if item['category'] == 'PCP':
        try:
            weather_data['rain'] += [item['fcstValue']]
        except:
            weather_data['rain'] = [item['fcstValue']]
    # 강수확률
    if item['category'] == 'POP':
        try:
            weather_data['rain_prb'] += [item['fcstValue']]
        except:
            weather_data['rain_prb'] = [item['fcstValue']]
    # 강수형태 (단기) 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4) 
    if item['category'] == 'PTY':
        try:
            weather_data['rain_type'] += [item['fcstValue']]
        except:
            weather_data['rain_type'] = [item['fcstValue']]
            
weather_data['tmp_low'] = weather_data['tmp_low']*len(weather_data["tmp_hour"])
weather_data['tmp_high'] = weather_data['tmp_high']*len(weather_data["tmp_hour"])
print("response: ", weather_data)

response:  {'fcstDate': ['20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704'], 'fcstTime': ['0000', '0100', '0200', '0300', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300'], 'tmp_hour': ['27', '26', '26', '25', '25', '25', '25', '25', '26', '27', '28', '28', '29', '29', '29', '28', '27', '27', '26', '26', '26', '26', '26', '25'], 'sky': ['1', '1', '1', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4'], 'rain_type': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1'], 'rain_prb': ['0', '0', '0', '20', '20', '30', '30', '30', 

In [ ]:
# 위도 경도를 X,Y 좌표로 바꾸기

import math
NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI


    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lon, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lon * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

def gridToMap(x, y, code = 1):
    x = x - 1
    y = y - 1
    xn = x - xo
    yn = ro - y + yo
    ra = math.sqrt(xn * xn + yn * yn)
    if sn < 0.0 :
        ra = -ra
    alat = math.pow((re * sf / ra), (1.0 / sn))
    alat = 2.0 * math.atan(alat) - PI * 0.5
    if math.fabs(xn) <= 0.0 :
        theta = 0.0
    else :
        if math.fabs(yn) <= 0.0 :
            theta = PI * 0.5
            if xn < 0.0 :
                theta = -theta
        else :
            theta = math.atan2(xn, yn)
    alon = theta / sn + olon
    lat = alat * RADDEG
    lon = alon * RADDEG

    return lat, lon

In [223]:
weather_data.values()

dict_values([['20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704', '20230704'], ['0000', '0100', '0200', '0300', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300'], ['27', '26', '26', '25', '25', '25', '25', '25', '26', '27', '28', '28', '29', '29', '29', '28', '27', '27', '26', '26', '26', '26', '26', '25'], ['1', '1', '1', '3', '3', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1'], ['0', '0', '0', '20', '20', '30', '30', '30', '30', '30', '30', '30', '30', '30', '60', '69', '64', '60', '30', '

In [230]:
# 키 별 값의 개수 구하기 
for j,k in weather_data.items():
    print(j, len(k))

fcstDate 24
fcstTime 24
tmp_hour 24
sky 24
rain_type 24
rain_prb 24
rain 24
hum 24
tmp_low 24
tmp_high 24


In [213]:
# 키 값 확인하기
for i in range(12):
    print(items["item"][i]["category"])

TMP
UUU
VVV
VEC
WSD
SKY
PTY
POP
WAV
PCP
REH
SNO


In [231]:
# 기상 데이터프레임화
import pandas as pd
weather_data_df=pd.DataFrame(weather_data)

In [233]:
import pickle
with open("weather_data_df", "wb") as f:
    pickle.dump(weather_data_df, f)
